In [2]:
# ==========================================
# 1. 환경 설정 및 경로 지정
# ==========================================
import sys
import os
from google.colab import drive

drive.mount('/content/drive')

# 본인의 경로에 맞게 수정
PROJECT_PATH = '/content/drive/MyDrive/DL_Project'
os.chdir(PROJECT_PATH)
sys.path.append(PROJECT_PATH)

# 제공된 common 라이브러리 임포트
try:
    from common.np import * # config.GPU에 따라 numpy 또는 cupy가 로드됨
    from common.time_layers import TimeEmbedding, TimeLSTM
    from common.layers import Affine, SoftmaxWithLoss
    from common.optimizer import Adam
    from common.trainer import Trainer
    from utils import extract_utterances
    import pandas as pd
    from collections import Counter
    print("✅ 라이브러리 로드 성공. 사용 모듈:", np.__name__)
except ImportError as e:
    print(f"❌ 라이브러리 로드 실패: {e}")
    print("폴더 구조가 올바른지 확인해주세요.")

# ==========================================
# 2. 데이터 전처리 및 로더 (Numpy 스타일)
# ==========================================
def load_vocab(train_csv, max_vocab=3000):
    df = pd.read_csv(train_csv)
    words = []
    print("⏳ Vocab 생성 중...")

    for _, row in df.iterrows():
        try:
            utts = extract_utterances(row['filename'], ['CHI'])
            text = " ".join([u.clean_text for u in utts]).lower()
            words.extend(text.split())
        except:
            continue

    counter = Counter(words)
    most_common = counter.most_common(max_vocab)

    vocab = {'<pad>': 0, '<unk>': 1}
    for i, (w, _) in enumerate(most_common):
        vocab[w] = i + 2

    print(f"✅ Vocab 크기: {len(vocab)}")
    return vocab

def load_data(csv_path, vocab, max_len=300):
    df = pd.read_csv(csv_path)
    x_data = []
    t_data = []

    label_map = {'TD': 0, 'SLI': 1}

    print(f"📂 데이터 로딩: {csv_path}")

    for _, row in df.iterrows():
        try:
            utts = extract_utterances(row['filename'], ['CHI'])
            if not utts: continue

            text = " ".join([u.clean_text for u in utts]).lower()
            tokens = text.split()

            # 정수 인코딩
            ids = [vocab.get(w, vocab['<unk>']) for w in tokens]

            # 길이 제한
            if len(ids) > max_len:
                ids = ids[:max_len]
            # 패딩 (Pre-padding or Post-padding)
            # 여기서는 편의상 Post-padding (뒤에 0 채움)
            # 실제 길이를 기억했다가 사용하는 것이 좋으나 간단한 구현을 위해 패딩만 수행
            padded_ids = np.zeros(max_len, dtype=int)
            length = min(len(ids), max_len)
            padded_ids[:length] = ids[:length]

            x_data.append(padded_ids)
            t_data.append(label_map[row['group']])

        except Exception as e:
            # print(f"Skip: {e}")
            pass

    return np.array(x_data), np.array(t_data)

# ==========================================
# 3. 모델 정의 (Rnnlm 변형 - 분류 모델)
# ==========================================
# ==========================================
# 3. 모델 정의 (수정됨)
# ==========================================
class SLPClassifier:
    def __init__(self, vocab_size, wordvec_size, hidden_size, class_size=2):
        V, D, H, C = vocab_size, wordvec_size, hidden_size, class_size
        rn = np.random.randn

        # 가중치 초기화 (Xavier)
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(H, C) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(C).astype('f')

        # 계층 생성
        self.layers = [
            TimeEmbedding(embed_W),
            TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False),
        ]
        self.affine_layer = Affine(affine_W, affine_b)
        self.loss_layer = SoftmaxWithLoss()

        # 모든 가중치와 기울기 수집
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

        self.params += self.affine_layer.params
        self.grads += self.affine_layer.grads

    def predict(self, xs):
        for layer in self.layers:
            xs = layer.forward(xs)

        # xs shape: (N, T, H) -> 마지막 시점의 은닉 상태만 사용
        h_last = xs[:, -1, :]
        score = self.affine_layer.forward(h_last)
        return score

    def forward(self, xs, ts):
        score = self.predict(xs)
        loss = self.loss_layer.forward(score, ts)
        return loss

    def backward(self, dout=1):
        # 1. Loss -> Affine 역전파
        dout = self.loss_layer.backward(dout)
        dout = self.affine_layer.backward(dout)

        # dout shape: (N, H)
        N, H = dout.shape

        # 2. TimeLSTM으로 넘겨줄 기울기(dhs) 생성
        # TimeEmbedding 계층(self.layers[0])이 보관 중인 layer 개수로 T를 추론
        T = len(self.layers[0].layers)

        # dhs shape: (N, T, H) 초기화
        dhs = np.zeros((N, T, H), dtype='f')

        # 분류 문제(Many-to-One)이므로 마지막 시점에만 기울기 주입
        dhs[:, -1, :] = dout

        # 3. LSTM -> Embedding 역전파
        for layer in reversed(self.layers):
            dhs = layer.backward(dhs)

        return dhs

# ==========================================
# 4. 데이터 로드 및 실행
# ==========================================
# 파일 경로
train_csv = 'split/gillam_train.csv'
dev_csv = 'split/gillam_dev.csv'
test_csv = 'split/gillam_test.csv'

# Vocab 생성
vocab = load_vocab(train_csv)
vocab_size = len(vocab)

# 데이터 로드 (Numpy Array)
x_train, t_train = load_data(train_csv, vocab)
x_dev, t_dev = load_data(dev_csv, vocab)
x_test, t_test = load_data(test_csv, vocab)

# 데이터를 CuPy Array로 변환 (GPU 모드일 경우)
if np.__name__ == 'cupy':
    x_train, t_train = np.asarray(x_train), np.asarray(t_train)
    x_dev, t_dev = np.asarray(x_dev), np.asarray(t_dev)
    x_test, t_test = np.asarray(x_test), np.asarray(t_test)

print(f"Train Shape: {x_train.shape}, {t_train.shape}")

# ==========================================
# 5. 학습 (Custom Loop)
# ==========================================
# 하이퍼파라미터
max_epoch = 10
batch_size = 16
hidden_size = 128
wordvec_size = 100
learning_rate = 0.001

# 모델 초기화
model = SLPClassifier(vocab_size, wordvec_size, hidden_size)
optimizer = Adam(lr=learning_rate)

# 학습 루프
data_size = len(x_train)
max_iters = data_size // batch_size

print(f"\n🚀 학습 시작 (Device: {np.__name__})")

for epoch in range(max_epoch):
    # 데이터 셔플
    idx = np.random.permutation(data_size)
    x_train = x_train[idx]
    t_train = t_train[idx]

    total_loss = 0
    loss_count = 0

    for i in range(max_iters):
        batch_x = x_train[i*batch_size : (i+1)*batch_size]
        batch_t = t_train[i*batch_size : (i+1)*batch_size]

        # 기울기 구하기
        loss = model.forward(batch_x, batch_t)
        model.backward()

        # 매개변수 갱신
        optimizer.update(model.params, model.grads)

        total_loss += loss
        loss_count += 1

    # 에폭마다 검증 (Accuracy)
    model_score = model.predict(x_dev)
    pred = np.argmax(model_score, axis=1)
    acc = np.sum(pred == t_dev) / len(t_dev)

    avg_loss = total_loss / loss_count
    print(f"| Epoch {epoch+1} | Loss {avg_loss:.4f} | Dev Acc {acc:.4f}")

# ==========================================
# 6. 최종 테스트
# ==========================================
test_score = model.predict(x_test)
test_pred = np.argmax(test_score, axis=1)
test_acc = np.sum(test_pred == t_test) / len(t_test)

print(f"\n🏆 Final Test Accuracy: {test_acc*100:.2f}%")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 라이브러리 로드 성공. 사용 모듈: numpy
⏳ Vocab 생성 중...
✅ Vocab 크기: 3002
📂 데이터 로딩: split/gillam_train.csv
📂 데이터 로딩: split/gillam_dev.csv
📂 데이터 로딩: split/gillam_test.csv
Train Shape: (540, 300), (540,)

🚀 학습 시작 (Device: numpy)
| Epoch 1 | Loss 0.6069 | Dev Acc 0.7353
| Epoch 2 | Loss 0.5235 | Dev Acc 0.7353
| Epoch 3 | Loss 0.4686 | Dev Acc 0.7500
| Epoch 4 | Loss 0.4011 | Dev Acc 0.7353
| Epoch 5 | Loss 0.3520 | Dev Acc 0.7500
| Epoch 6 | Loss 0.3164 | Dev Acc 0.7353
| Epoch 7 | Loss 0.2875 | Dev Acc 0.7206
| Epoch 8 | Loss 0.2948 | Dev Acc 0.7206
| Epoch 9 | Loss 0.2595 | Dev Acc 0.7059
| Epoch 10 | Loss 0.2722 | Dev Acc 0.7353

🏆 Final Test Accuracy: 70.59%
